# DSCI 552 — Homework 3
**Student:** Yudie Deng  
**GitHub Username:** _yudiedeng_  
**USC ID:** _7162812062_


## 1. Time Series Classification — Part 1: Feature Creation / Extraction

### 1(a) Data
Place the unzipped **AReM** folders under `../data/AReM/`. Each subfolder (e.g., `bending1`) contains multiple instance files.

In [ ]:
#i find that some datasets are not in the correct format, so i need to modify the code to load the data
#i use the chatgpt to help me modify the code(prompt: i want to load the data from the csv file, and the data is not in the correct format, so i need to modify the code to load the data)
import os, glob
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

EXPECTED_COLS = 6

def load_arem_super_simple(base_dir):
    X, y_str, lengths, problems = [], [], [], []

    for cls in sorted(os.listdir(base_dir)):
        cls_dir = os.path.join(base_dir, cls)
        if not os.path.isdir(cls_dir):
            continue

        for fp in sorted(glob.glob(os.path.join(cls_dir, "*.csv"))):
            rel = f"{cls}/{os.path.basename(fp)}"
            try:
                # try use comma
                df = pd.read_csv(fp, comment='#', header=None)
                
                # try use space
                if df.shape[1] < EXPECTED_COLS:
                    df = pd.read_csv(fp, comment='#', header=None, sep=r'\s+')
                

                # check final columns
                if df.shape[1] < EXPECTED_COLS:
                    problems.append({"file": rel, "reason": f"only {df.shape[1]} columns after all attempts"})
                    continue
                    
                # take last 6 columns
                df = df.iloc[:, -EXPECTED_COLS:]

                # convert to numeric and drop all-NaN rows
                df = df.apply(pd.to_numeric, errors="coerce").dropna(how="all")

                if df.shape[0] == 0:
                    problems.append({"file": rel, "reason": "empty after cleaning"})
                    continue

                X.append(df.to_numpy(dtype=float))
                y_str.append(cls)
                lengths.append(df.shape[0])

            except Exception as e:
                problems.append({"file": rel, "reason": str(e)})

    if not X:
        raise RuntimeError("No valid samples loaded")

    le = LabelEncoder()
    y = le.fit_transform(y_str)
    class_names = list(le.classes_)
    lengths = np.array(lengths)


    if problems:
        print("\n[issues] Problematic files:")
        for p in problems:
            print(f" - {p['file']}: {p['reason']}")

    return X, y, class_names, lengths, problems


### 1(b) Train/Test Split by Folder Rules
- Test: datasets 1 & 2 in `bending1` and `bending2`; datasets 1, 2, 3 in other folders.
- Train: remaining datasets.

In [26]:

X_train, y_train, class_names, len_train, probs_train = load_arem_super_simple("../data/AReM_split/train/")
X_test,  y_test,  _,           len_test,  probs_test  = load_arem_super_simple("../data/AReM_split/test/")

print(f"Train samples: {len(X_train)}, Test samples: {len(X_test)}")
print("Class names:", class_names)
print("Train length range:", len_train.min(), "to", len_train.max())
print("Test length range:", len_test.min(), "to", len_test.max())

# print problematic files
if probs_train or probs_test:
    print("\n[Problem Files]")
    for p in probs_train + probs_test:
        print(f" - {p['file']}: {p['reason']}")
else:
    print("\n All train/test files successfully loaded!")


Train samples: 69, Test samples: 19
Class names: [np.str_('bending1'), np.str_('bending2'), np.str_('cycling'), np.str_('lying'), np.str_('sitting'), np.str_('standing'), np.str_('walking')]
Train length range: 479 to 480
Test length range: 480 to 480

 All train/test files successfully loaded!


### 1(c) Feature Extraction
**Common time-domain features** (not exhaustive): minimum, maximum, mean, median, standard deviation, variance, range, interquartile range (IQR), first quartile (Q1), third quartile (Q3), skewness, kurtosis, energy, zero-crossing rate, mean absolute deviation, peak-to-peak amplitude.

We compute: minimum, maximum, mean, median, standard deviation, Q1, Q3 for each of the six series per instance.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# i told the chatgpt that i want a big loop outside and a small loop inside to help me use numpy to calculate the features
def extract_time_features_with_instance(X):
    all_features = []
    for idx, sample in enumerate(X, start=1):
        row_feats = [idx]  # Instance 
        for j in range(sample.shape[1]):
            x = sample[:, j]
            row_feats.extend([
                np.min(x), np.max(x),
                np.mean(x), np.median(x),
                np.std(x),
                np.percentile(x, 25), np.percentile(x, 75)
            ])
        all_features.append(row_feats)

    # print column names
    colnames = ["Instance"]
    for j in range(1, 7):
        colnames += [
            f"min{j}", f"max{j}", f"mean{j}", f"median{j}",
            f"std{j}", f"1st_quart{j}", f"3rd_quart{j}"
        ]

    return pd.DataFrame(all_features, columns=colnames)

X_train_features = extract_time_features_with_instance(X_train)
X_test_features  = extract_time_features_with_instance(X_test)

print("Train feature table:")
print(X_train_features.head())


Train feature table:
   Instance   min1   max1      mean1  median1      std1  1st_quart1  \
0         1  35.00  47.40  43.954500    44.33  1.557210       43.00   
1         2  33.00  47.75  42.179812    43.50  3.666840       39.15   
2         3  33.00  45.75  41.678063    41.75  2.241152       41.33   
3         4  37.00  48.00  43.454958    43.25  1.384653       42.50   
4         5  36.25  48.00  43.969125    44.50  1.616677       43.31   

   3rd_quart1  min2  max2  ...      std5  1st_quart5  3rd_quart5  min6  max6  \
0       45.00   0.0  1.70  ...  1.997520     35.3625       36.50   0.0  1.79   
1       45.00   0.0  3.00  ...  3.845436     30.4575       36.33   0.0  2.18   
2       42.75   0.0  2.83  ...  2.408514     28.4575       31.25   0.0  1.79   
3       45.00   0.0  1.58  ...  2.486268     22.2500       24.00   0.0  5.26   
4       44.67   0.0  1.50  ...  3.314843     20.5000       23.75   0.0  2.96   

      mean6  median6      std6  1st_quart6  3rd_quart6  
0  0.493292   

In [ ]:
# calculate standard deviation with bootstrap CI
#i told the chatgpt that i need the loop outside and use the bootstrap to calculate the CI
import pandas as pd
from scipy.stats import bootstrap

def bootstrap_std_ci_scipy(df, ci=0.90, n_resamples=2000, random_state=42):
    results = []
    for col in df.columns:
        data = df[col].dropna().to_numpy()
        if len(data) < 2:
            results.append((col, np.nan, np.nan, np.nan))
            continue

        res = bootstrap(
            (data,), 
            np.std,
            confidence_level=ci,
            n_resamples=n_resamples,
            method="percentile",
            random_state=random_state
        )
        std_hat = np.std(data, ddof=1)
        ci_low = res.confidence_interval.low
        ci_high = res.confidence_interval.high
        results.append((col, std_hat, ci_low, ci_high))

    return pd.DataFrame(results, columns=["feature", "std_hat", "ci_lower", "ci_upper"])

ci90_df = bootstrap_std_ci_scipy(X_train_features.iloc[:, 1:], ci=0.90)
print(ci90_df)

       feature   std_hat  ci_lower   ci_upper
0         min1  8.794295  7.544201  10.035101
1         max1  4.429182  3.257113   5.341745
2        mean1  4.917717  4.316704   5.386538
3      median1  4.956111  4.307741   5.435978
4         std1  1.756796  1.537618   1.933586
5   1st_quart1  5.731262  5.123539   6.139473
6   3rd_quart1  4.783645  3.920147   5.480000
7         min2  0.000000  0.000000   0.000000
8         max2  5.147841  4.611797   5.488066
9        mean2  1.600661  1.397702   1.732134
10     median2  1.436903  1.235323   1.573123
11        std2  0.901829  0.801854   0.955654
12  1st_quart2  0.952201  0.818942   1.047242
13  3rd_quart2  2.158258  1.889708   2.325299
14        min3  3.053869  2.817137   3.200269
15        max3  4.759853  3.949477   5.370447
16       mean3  3.863304  3.186092   4.382022
17     median3  3.845730  3.148523   4.403001
18        std3  0.994970  0.770499   1.190774
19  1st_quart3  4.145255  3.464232   4.671244
20  3rd_quart3  3.946023  3.240262

#### 1(c)(iv) Choose Three Features
I select: **min, mean, max**.

- Min/Max capture the amplitude envelope and extrema, which differ across activities (e.g., dynamic motions like walking tend to have higher variability and broader extrema than static postures like sitting/lying).
- Mean captures the central tendency (baseline level), which can shift across activities and sensors.

These three are simple, robust, and complementary: extrema summarize range, mean summarizes level. They also tend to be less sensitive to small sample peculiarities than higher-order moments.

## 2. Linear vs Cubic Regression Analysis

I collect a set of data (n = 100 observations) containing a single predictor and a quantitative response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e. Y = β₀ + β₁X + β₂X² + β₃X³ + ϵ.



In [29]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
np.random.seed(42)


In [30]:
# Helper functions for data generation and model fitting
def generate_data(n=100, true_relationship='linear', noise_level=1.0):
    """Generate data with specified true relationship"""
    X = np.linspace(-2, 2, n).reshape(-1, 1)
    
    if true_relationship == 'linear':
        y_true = 2 * X.flatten() + 1
    elif true_relationship == 'quadratic':
        y_true = X.flatten()**2 + 0.5 * X.flatten() + 1
    elif true_relationship == 'cubic':
        y_true = 0.5 * X.flatten()**3 - X.flatten()**2 + 2 * X.flatten() + 1
    else:
        y_true = np.sin(2 * X.flatten()) + 0.5 * X.flatten()
    
    # Add noise
    y = y_true + noise_level * np.random.randn(n)
    return X, y, y_true

def fit_models(X, y):
    """Fit linear and cubic regression models"""
    # Linear regression
    linear_model = LinearRegression()
    linear_model.fit(X, y)
    y_pred_linear = linear_model.predict(X)
    
    # Cubic regression
    poly_features = PolynomialFeatures(degree=3, include_bias=False)
    X_poly = poly_features.fit_transform(X)
    cubic_model = LinearRegression()
    cubic_model.fit(X_poly, y)
    y_pred_cubic = cubic_model.predict(X_poly)
    
    return linear_model, cubic_model, y_pred_linear, y_pred_cubic

def calculate_rss(y_true, y_pred):
    """Calculate Residual Sum of Squares"""
    return np.sum((y_true - y_pred)**2)


### 2(a) Training RSS when true relationship is linear

**Question:** Suppose that the true relationship between X and Y is linear, i.e. Y = β₀ + β₁X + ϵ. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

**Answer:** We would expect the **cubic regression to have lower or equal training RSS** compared to the linear regression.

**Justification:**
- The cubic model includes the linear model as a special case (when β₂ = β₃ = 0)
- Since the cubic model has more parameters (4 vs 2), it has more flexibility to fit the training data
- Even if the true relationship is linear, the cubic model can still fit the data perfectly by setting β₂ ≈ 0 and β₃ ≈ 0
- The linear model is constrained to a linear relationship, while the cubic model can approximate any smooth function
- Therefore: RSS_cubic ≤ RSS_linear (with equality only if the linear fit is already optimal)

### 2(b) Test RSS when true relationship is linear

**Question:** Answer (a) using test rather than training RSS.

**Answer:** We would expect the **linear regression to have lower test RSS** compared to the cubic regression.

**Justification:**
- Since the true relationship is linear, the cubic model will likely overfit to the training data
- The cubic model's additional parameters (β₂, β₃) will capture noise rather than true signal
- On new test data, the overfitted cubic model will perform worse than the correctly specified linear model
- The linear model is the correct specification, so it will generalize better to unseen data
- Therefore: RSS_cubic_test > RSS_linear_test


In [31]:
# Demonstrate with linear relationship (true relationship is linear)
print("=== Linear Relationship Demo ===")
X, y, y_true = generate_data(n=100, true_relationship='linear', noise_level=0.5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit models
linear_model, cubic_model, y_pred_linear_train, y_pred_cubic_train = fit_models(X_train, y_train)

# Test predictions
y_pred_linear_test = linear_model.predict(X_test)
poly_features = PolynomialFeatures(degree=3, include_bias=False)
X_test_poly = poly_features.fit_transform(X_test)
y_pred_cubic_test = cubic_model.predict(X_test_poly)

# Calculate RSS
rss_linear_train = calculate_rss(y_train, y_pred_linear_train)
rss_cubic_train = calculate_rss(y_train, y_pred_cubic_train)
rss_linear_test = calculate_rss(y_test, y_pred_linear_test)
rss_cubic_test = calculate_rss(y_test, y_pred_cubic_test)

print(f"Training RSS - Linear: {rss_linear_train:.2f}, Cubic: {rss_cubic_train:.2f}")
print(f"Test RSS - Linear: {rss_linear_test:.2f}, Cubic: {rss_cubic_test:.2f}")
print(f"Cubic ≤ Linear on training: {rss_cubic_train <= rss_linear_train}")
print(f"Linear < Cubic on test: {rss_linear_test < rss_cubic_test}")


=== Linear Relationship Demo ===
Training RSS - Linear: 16.23, Cubic: 15.20
Test RSS - Linear: 4.16, Cubic: 4.19
Cubic ≤ Linear on training: True
Linear < Cubic on test: True


### 2(c) Training RSS when true relationship is unknown

**Question:** Suppose that the true relationship between X and Y is not linear, but we don't know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

**Answer:** We would expect the **cubic regression to have lower training RSS** compared to the linear regression.

**Justification:**
- The cubic model has more flexibility and can capture non-linear relationships
- If the true relationship is non-linear, the linear model will be misspecified and have higher bias
- The cubic model can approximate many smooth non-linear functions better than a linear model
- With more parameters, the cubic model has more capacity to fit the training data
- Therefore: RSS_cubic < RSS_linear (unless the relationship is actually linear)

### 2(d) Test RSS when true relationship is unknown

**Question:** Answer (c) using test rather than training RSS.

**Answer:** **There is not enough information to tell** which will have lower test RSS.

**Justification:**
- The answer depends on the **bias-variance tradeoff**:
  - **Linear model**: Higher bias (if relationship is non-linear) but lower variance
  - **Cubic model**: Lower bias (can capture non-linear patterns) but higher variance
- **If the relationship is close to linear**: Linear model wins (lower test RSS)
- **If the relationship is highly non-linear**: Cubic model wins (lower test RSS)  
- **If the relationship is moderately non-linear**: Depends on the specific form and noise level
- **Sample size matters**: With n=100, the cubic model might overfit even if the relationship is non-linear
- **Noise level matters**: High noise favors simpler models (linear), low noise favors more complex models (cubic)

**Conclusion:** Without knowing the specific form of the non-linear relationship, we cannot definitively say which model will perform better on test data.


In [32]:
# Demonstrate with non-linear relationship
print("\n=== Non-linear Relationship Demo ===")
X, y, y_true = generate_data(n=100, true_relationship='cubic', noise_level=0.5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit models
linear_model, cubic_model, y_pred_linear_train, y_pred_cubic_train = fit_models(X_train, y_train)

# Test predictions
y_pred_linear_test = linear_model.predict(X_test)
poly_features = PolynomialFeatures(degree=3, include_bias=False)
X_test_poly = poly_features.fit_transform(X_test)
y_pred_cubic_test = cubic_model.predict(X_test_poly)

# Calculate RSS
rss_linear_train = calculate_rss(y_train, y_pred_linear_train)
rss_cubic_train = calculate_rss(y_train, y_pred_cubic_train)
rss_linear_test = calculate_rss(y_test, y_pred_linear_test)
rss_cubic_test = calculate_rss(y_test, y_pred_cubic_test)

print(f"Training RSS - Linear: {rss_linear_train:.2f}, Cubic: {rss_cubic_train:.2f}")
print(f"Test RSS - Linear: {rss_linear_test:.2f}, Cubic: {rss_cubic_test:.2f}")
print(f"Cubic < Linear on training: {rss_cubic_train < rss_linear_train}")
print(f"Cubic < Linear on test: {rss_cubic_test < rss_linear_test}")



=== Non-linear Relationship Demo ===
Training RSS - Linear: 143.16, Cubic: 15.96
Test RSS - Linear: 72.92, Cubic: 6.52
Cubic < Linear on training: True
Cubic < Linear on test: True
